# Data Prep for Raw Stats

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv('FG_2010_2018_inning_6_raw.csv')

In [3]:
df.head(1)

,Season,Name,Team,G,Age,PA,HR,R,RBI,SB,...,Off,O-Swing%,Z-Swing%,O-Contact%,Z-Contact%,GB/FB,LD%,GB%,HR/FB,playerid
0,2018,Mookie Betts,Red Sox,136,25,614,32,129,80,30,...,69.2,19.80%,57.00%,70.70%,93.00%,0.75,21.20%,33.90%,16.40%,13611


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1858 entries, 0 to 1857
Data columns (total 31 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Season      1858 non-null   int64  
 1   Name        1858 non-null   object 
 2   Team        1858 non-null   object 
 3   G           1858 non-null   int64  
 4   Age         1858 non-null   int64  
 5   PA          1858 non-null   int64  
 6   HR          1858 non-null   int64  
 7   R           1858 non-null   int64  
 8   RBI         1858 non-null   int64  
 9   SB          1858 non-null   int64  
 10  BB%         1858 non-null   object 
 11  K%          1858 non-null   object 
 12  ISO         1858 non-null   float64
 13  BABIP       1858 non-null   float64
 14  AVG         1858 non-null   float64
 15  OBP         1858 non-null   float64
 16  SLG         1858 non-null   float64
 17  wOBA        1858 non-null   float64
 18  wRC+        1858 non-null   int64  
 19  EV          844 non-null   

In [5]:
# find percent strings and change to floats
for x in ['BB%', 'K%', 'O-Swing%', 'Z-Swing%','O-Contact%',
          'Z-Contact%','LD%','GB%','HR/FB']:
    df[x] = df[x].str.rstrip('%').astype('float') / 100.0

## Calculate rates for counting stats

In [6]:
df['HR_rate']= df['HR'].div(df['G'])
df['R_rate']= df['R'].div(df['G'])
df['RBI_rate']= df['RBI'].div(df['G'])
df['SB_rate']= df['SB'].div(df['G'])

In [7]:
df.head(1)

,Season,Name,Team,G,Age,PA,HR,R,RBI,SB,...,Z-Contact%,GB/FB,LD%,GB%,HR/FB,playerid,HR_rate,R_rate,RBI_rate,SB_rate
0,2018,Mookie Betts,Red Sox,136,25,614,32,129,80,30,...,0.93,0.75,0.212,0.339,0.164,13611,0.235294,0.948529,0.588235,0.220588


## Standardize Rates (uses as individual stat FV)

#### First create tables of standard devs and means

In [8]:
seasons = df['Season'].unique()
stat_cols = ['HR_rate','R_rate','RBI_rate','SB_rate','AVG'] 

In [9]:
standard_devs = pd.DataFrame(data=[],index=seasons, columns=stat_cols)
means = pd.DataFrame(data=[],index=seasons, columns=stat_cols)  
for year in seasons:
    for col in stat_cols:
        standard_devs.loc[year,col] = df[df['Season']==year][col].std(ddof=0) #population stdev since all players are accounted for
        means.loc[year,col] = df[df['Season']==year][col].mean()

#### Iterate through table and standardize rates

In [10]:
for i in list(df.index):
    for col in stat_cols:
        year = df.loc[i,'Season']
        df.loc[i,col+'_std'] = (df.loc[i,col]-means.loc[year, col])/standard_devs.loc[year, col]

In [11]:
df

,Season,Name,Team,G,Age,PA,HR,R,RBI,SB,...,playerid,HR_rate,R_rate,RBI_rate,SB_rate,HR_rate_std,R_rate_std,RBI_rate_std,SB_rate_std,AVG_std
0,2018,Mookie Betts,Red Sox,136,25,614,32,129,80,30,...,13611,0.235294,0.948529,0.588235,0.220588,1.650655,4.013682,0.896560,2.800255,3.154872
1,2018,Mike Trout,Angels,140,26,608,39,101,79,24,...,10155,0.278571,0.721429,0.564286,0.171429,2.361526,2.006636,0.706851,1.969550,1.909222
2,2018,J.D. Martinez,Red Sox,150,30,649,43,111,130,6,...,6184,0.286667,0.740000,0.866667,0.040000,2.494498,2.170765,3.102064,-0.251343,2.568684
3,2018,Christian Yelich,Brewers,147,26,651,36,118,110,22,...,11477,0.244898,0.802721,0.748299,0.149660,1.808407,2.725074,2.164455,1.601700,2.422137
4,2018,Max Muncy,Dodgers,137,27,481,35,75,79,3,...,13301,0.255474,0.547445,0.576642,0.021898,1.982136,0.469027,0.804730,-0.557236,0.114019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1853,2010,Alcides Escobar,Brewers,145,23,552,4,57,41,10,...,6310,0.027586,0.393103,0.282759,0.068966,-1.307708,-0.953568,-1.369961,-0.064176,-1.392512
1854,2010,Jose Lopez,Mariners,150,26,622,10,49,58,3,...,3114,0.066667,0.326667,0.386667,0.020000,-0.708731,-1.539959,-0.639477,-0.671665,-1.236007
1855,2010,Brendan Ryan,Cardinals,139,28,486,2,50,36,11,...,6073,0.014388,0.359712,0.258993,0.079137,-1.509987,-1.248289,-1.537037,0.062013,-1.862026
1856,2010,Cesar Izturis,Orioles,150,30,513,1,42,28,11,...,656,0.006667,0.280000,0.186667,0.073333,-1.628338,-1.951853,-2.045497,-0.009986,-1.588143


## Sum up standardized rates to create fantasy value

In [12]:
df['f_val_std'] = df[['HR_rate_std', 'R_rate_std', 'RBI_rate_std','SB_rate_std', 'AVG_std']].sum(axis=1)

## Append the next season's f_val to players to be used as target

In [13]:
# create a copy of the data frame to adjust the season down one
temp = df.copy()
temp['Season'] = temp['Season'] - 1
temp = temp[['Season','playerid','f_val_std']]

In [14]:
# merge the shifted f_val to the original df, rename the cols, drop players that did not qualify the next year, reset index
df_output = pd.merge(df, temp,  how='left', left_on=['Season','playerid'], right_on = ['Season','playerid'])
df_output.rename(columns={'f_val_std_x':'f_val_std', 'f_val_std_y': 'ny_f_val'}, inplace=True)
df_output.dropna(axis=0, subset=['ny_f_val'], inplace=True)
df_output.reset_index(drop=True, inplace=True)

In [15]:
df_output

,Season,Name,Team,G,Age,PA,HR,R,RBI,SB,...,R_rate,RBI_rate,SB_rate,HR_rate_std,R_rate_std,RBI_rate_std,SB_rate_std,AVG_std,f_val_std,ny_f_val
0,2017,Mike Trout,Angels,114,25,507,33,92,72,22,...,0.807018,0.631579,0.192982,2.061054,2.497973,0.957319,1.866089,1.364350,8.746785,8.953784
1,2017,J.D. Martinez,- - -,119,29,489,45,85,104,4,...,0.714286,0.873950,0.033613,3.387659,1.682132,2.818576,-0.372138,1.260631,8.776860,10.084667
2,2017,Aaron Judge,Yankees,155,25,678,52,128,114,9,...,0.825806,0.735484,0.058065,2.749360,2.663275,1.755245,-0.028740,0.603746,7.742886,5.069503
3,2017,Joey Votto,Reds,162,33,707,36,106,100,5,...,0.654321,0.617284,0.030864,1.054981,1.154571,0.847542,-0.410749,1.848371,4.494716,-1.054013
4,2017,Bryce Harper,Nationals,111,24,492,29,95,87,4,...,0.855856,0.783784,0.036036,1.639000,2.927645,2.126158,-0.338114,1.813798,8.168487,3.928470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1149,2010,Ryan Theriot,- - -,150,30,640,2,72,29,20,...,0.480000,0.193333,0.133333,-1.526159,-0.186593,-1.998629,0.734401,-0.023094,-3.000074,-4.276138
1150,2010,Clint Barmes,Rockies,133,31,432,8,43,50,3,...,0.323308,0.375940,0.022556,-0.808605,-1.569601,-0.714887,-0.639949,-1.392512,-5.125553,-3.895471
1151,2010,Casey Kotchman,Mariners,125,27,457,9,37,51,0,...,0.296000,0.408000,0.000000,-0.626988,-1.810632,-0.489501,-0.919794,-2.096784,-5.943699,-2.916351
1152,2010,Alcides Escobar,Brewers,145,23,552,4,57,41,10,...,0.393103,0.282759,0.068966,-1.307708,-0.953568,-1.369961,-0.064176,-1.392512,-5.087924,-2.582905


In [16]:
df_output.to_csv('prepared_baseball_data.csv', index=False)